# LunaZodiaco

LunaZodiaco is a F# NuGet package to calculate the moon phase and the moon's zodiac (ecliptic longitude) at a given day.
It uses [Conway's method](http://www.faqs.org/faqs/astronomy/faq/part3/section-15.html) to calculate the moon age, which works in the 20th and 21th century and has an accuracy of about 1 day. The zodiac / ecliptic longitude ist calculated using an algorithm of [Paul Schlyter](http://stjarnhimlen.se/comp/tutorial.html#7) - *Computing planetary positions - a tutorial with worked examples*.

So both values aren't accurate enough for use in a serious astronomy program, but suitable for a calender like application - the position changes during one day anyway.

## Installation of the NuGet Package

Download an install the LunaZodiaco Nuget package using your IDE (Visual Studio or JetBrains Rider) or one of the command-line possibilities: [LunaZodiaco Package at NuGet](https://www.nuget.org/packages/LunaZodiaco/)

In an interactive F# FSI session, you can use the command #r "nuget: LunaZodiaco" to download and use the NuGet package LunaZodiaco.

In [1]:
#r "nuget: LunaZodiaco"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package LunaZodiaco version 0.9.2

Everything is located in the namespace `RC.Moon`, so lets open that.

To check if everything is working, we try to display the Unicode codepoint of a full moon:

In [2]:
open RC.Moon

MoonPhase.FullMoon.ToUnicode ()

🌕️

## Module MoonPhase

Types and functions regarding the lunar phase are located in the module `MoonPhase`.

There is a discriminated union for all 8 lunar phases:

In [3]:
MoonPhase.NewMoon

Luna nueva

In [4]:
MoonPhase.WaxingCrescent

Luna creciente

In [5]:
MoonPhase.FirstQuarter

Cuarto creciente

In [6]:
MoonPhase.WaxingGibbous

Creciente gibosa

In [7]:
MoonPhase.FullMoon

Luna Llena

In [8]:
MoonPhase.WaningGibbous

Menguante gibosa

In [9]:
MoonPhase.LastQuarter

Cuarto menguante

In [10]:
MoonPhase.WaningCrescent

Luna menguante

The member `ToString` and the function `MoonPhase.toString` both return the Spanish name of the lunar phase.

In [11]:
MoonPhase.NewMoon.ToString ()

Luna nueva

In [12]:
MoonPhase.toString MoonPhase.NewMoon

Luna nueva

The member `ToUnicode` returns the Unicode codepoint of the lunar phase, a symbol. Same as the function `MoonPhase.toUnicode`.

In [13]:
MoonPhase.FirstQuarter.ToUnicode ()

🌓️

In [14]:
MoonPhase.toUnicode MoonPhase.FirstQuarter

🌓️

The member `ToInt` return an integer index of the lunar phase, where `0` is the new moon, `4` is the full moon and `7` waning crescent.

In [15]:
MoonPhase.WaxingGibbous.ToInt ()

3

The member `FromInt` and function `MoonPhase.fromInt` convert an int to a moon phase option, it is the inverse of the function `ToInt` above.

In [16]:
MoonPhase.fromInt 3

Value
Creciente gibosa


In [17]:
MoonPhase.T.FromInt 3

Value
Creciente gibosa


Too big or small integers yield `None`.

In [18]:
MoonPhase.fromInt -6

The function `MoonPhase.getInterval` and member `Interval` return the elongation interval of the phase.

The last quarter occures between an elongation angle (between the moon and the sun) of 270 degrees up to, but not including, 315 degrees.

In [19]:
MoonPhase.LastQuarter.Interval ()

Item1,Item2
270,315


In [20]:
MoonPhase.getInterval MoonPhase.LastQuarter

Item1,Item2
270,315


The inverse function of `getInterval` is `fromAngle`, the member function is named `FromAngle`. This returns the lunar phase to a given elongation angle. Angles in degrees need the measurement `<deg>`.

In [21]:
MoonPhase.fromAngle 346.<deg>

Value
Luna menguante


In [22]:
MoonPhase.T.FromAngle 346.<deg>

Value
Luna menguante


## Module Zodiac

All zodiac related functions the the zodiac type are contained in the module `Zodiac`.

There is a discriminated union for all 12 zodiacs:

In [23]:
Zodiac.Aries

Aries

In [24]:
Zodiac.Taurus

Tauro

In [25]:
Zodiac.Gemini

Géminis

And so on, until

In [26]:
Zodiac.Aquarius

Acuario

In [27]:
Zodiac.Pisces

Piscis

The member `ToString` and the function `Zodiac.toString` both return the Spanish name of the lunar phase.

In [28]:
Zodiac.Aries.ToString ()

Aries

In [29]:
Zodiac.toString Zodiac.Aries

Aries

The member `ToUnicode` returns the Unicode codepoint of the zodiac, a symbol. Same as the function `Zodiac.toUnicode`.

In [30]:
Zodiac.Gemini.ToUnicode ()

♊

In [31]:
Zodiac.toUnicode Zodiac.Gemini

♊

The member `ToInt` return an integer index of the zodiac, where `0` is the Aries, ... and `11` Pisces.

In [32]:
Zodiac.Capricorn.ToInt ()

9

The member `FromInt` and function `Zodiac.fromInt` convert an int to a moon phase option, it is the inverse of the function `ToInt` above.

In [33]:
Zodiac.fromInt 5

Value
Virgo


In [34]:
Zodiac.T.FromInt 5

Value
Virgo


Too big or small integers yield `None`.

In [35]:
Zodiac.fromInt -6

The function `Zodiac.getInterval` and member `Interval` return the ecliptic longitude interval of the phase.

Libra occurs in the ecliptic longitude angle of 180 to 210 degrees, including 180 but excluding 210.

In [36]:
Zodiac.Libra.Interval ()

Item1,Item2
180,210


In [37]:
Zodiac.getInterval Zodiac.Libra

Item1,Item2
180,210


The inverse function of `getInterval` is `fromAngle`, the member function is named `FromAngle`. This returns the zodiac to a given ecliptic longitude angle. Angles in degrees need the measurement `<deg>`.

In [38]:
Zodiac.fromAngle 346.<deg>

Value
Piscis


In [39]:
Zodiac.T.FromAngle 346.<deg>

Value
Piscis


## Module LunaZodiaco

This module contains the moon phase and zodiac calculations and the type `MoonDay`, that holds the zodiac and lunar phase for a given date.

In [40]:
let moonDay = { LunaZodiaco.MoonDay.Phase = MoonPhase.WaningGibbous; LunaZodiaco.Zodiac = Zodiac.Capricorn }
moonDay

Phase,Zodiac
Menguante gibosa,Capricornio


`MoonDay` does have the member `ToStrings` and function `LunaZodiaco.toStrings`, that return a tuple of strings

In [41]:
LunaZodiaco.toStrings moonDay

Item1,Item2
Menguante gibosa,Capricornio


In [42]:
moonDay.ToStrings ()

Item1,Item2
Menguante gibosa,Capricornio


And the member `ToString`, that returns a single string.

In [43]:
moonDay.ToString ()

{ Phase = WaningGibbous
  Zodiac = Capricorn }

The function `LunaZodiaco.toUnicode` returns the moon phase and zodiac as a tuple of Unicode code points / symbols:

In [44]:
LunaZodiaco.toUnicode moonDay

Item1,Item2
🌖️,♑


In [45]:
moonDay.ToUnicode ()

Item1,Item2
🌖️,♑


The member `ToIntervals` and the function `LunaZodiaco.toIntervals` return the elongation angle and the ecliptic longitude angle interval of the moon phase and zodiac as a tuple.

In [46]:
LunaZodiaco.toIntervals moonDay

Item1,Item2
"( 225, 270 )","( 270, 300 )"


In [47]:
moonDay.ToIntervals ()

Item1,Item2
"( 225, 270 )","( 270, 300 )"


## Calculation of Moon Age and Ecliptic Longitude

The function `LunaZodiaco.computeMoonPhase` returns the moon phase interval of the moon at a given date, with an accuracy of +/- one day. Attention: it only works in the 20th and 21st century, as it uses Conways algorithm to calculate the moon age (the day of the moon in one lunar cycle of 29 days).

In [48]:
open System
LunaZodiaco.computeMoonPhase (DateTime (2021, 05, 06))

Item1,Item2
315,360


THe function `LunaZodiaco.computeLongitude` calculates the ecliptic longitude of the moon at the given day (well, an angle that it has during the day).

In [49]:
LunaZodiaco.computeLongitude (DateTime (2021, 05, 06))

340.9314455776453

To get the moon phase and moon zodiac of a day, use the function `LunaZodiaco.getMoonDay`, that returns the moon phase and the zodiac as a `MoonDay` record.

In [50]:
LunaZodiaco.getMoonDay (DateTime (2021, 05, 06))

Phase,Zodiac
Luna menguante,Piscis


&copy; 2021 Roland Csaszar, licensed under the MIT license.